Regime Model Greedy Gaussian Segmentation of Multivariate Time Series

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
from GGS.ggs import *

In [12]:
import itertools
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.dates import YearLocator, MonthLocator
plt.rcParams["figure.figsize"] = (12, 8) # (width, height)

import sklearn.mixture as mix

from utils.index_data import index_data

In [13]:
data_setting = {
    'start' : '1992-01-01',
    'end'   : '2021-06-30',
    'rebal_period':'1M'
}

data = index_data(**data_setting)
return_df = data.index_data_loading()

In [14]:
return_df

,msci_acwi,msci_world,msci_emerging,msci_world_gross,msci_world_value,msci_real_estate,bb_world_agg,bb_emerging_agg,bb_corp_ig,bb_corp_hy,bb_infla_protect,snp_commodity
1992-01-31,0.000000,0.000000,0.000000,-0.031464,-0.009100,0.000000,-0.002171,0.000000,-0.002588,0.035225,0.000000,0.005368
1992-02-29,-0.017548,-0.020678,0.103012,-0.023750,-0.014170,0.000000,-0.013095,0.000000,-0.012103,0.024673,0.000000,0.025628
1992-03-31,-0.015102,-0.017357,0.054748,-0.048605,-0.049508,0.000000,0.003074,0.000000,0.006558,0.012399,0.000000,-0.023659
1992-04-30,-0.063123,-0.066034,0.029763,-0.011712,0.035704,0.000000,0.003851,0.000000,0.005622,0.003798,0.000000,0.030560
1992-05-31,0.026335,0.027579,-0.009451,0.042249,0.033392,0.000000,0.006528,0.000000,0.004180,0.014110,0.000000,0.023145
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-28,0.009655,0.002914,0.053679,0.009748,-0.003860,0.008924,-0.006128,-0.007934,-0.010356,0.004106,0.004246,0.068418
2021-03-31,0.027554,0.031810,0.001014,0.007317,0.056323,0.007271,-0.016604,-0.013898,-0.020684,0.005491,-0.020821,0.079945
2021-04-30,0.014839,0.020293,-0.019986,-0.001131,0.040848,0.049885,-0.007515,-0.010507,-0.008003,0.000708,0.000705,0.001453
2021-05-31,0.033565,0.038195,0.002734,0.044316,0.032582,0.048829,0.005114,0.009870,0.006059,0.010157,0.017477,0.075171


In [15]:
return_df = return_df.T
_return_df = np.array(return_df)
_return_df

array([[ 0.        , -0.01754782, -0.01510157, ...,  0.01483923,
         0.03356504,  0.01485831],
       [ 0.        , -0.02067767, -0.01735681, ...,  0.02029314,
         0.03819463,  0.01131018],
       [ 0.        ,  0.10301247,  0.05474808, ..., -0.01998576,
         0.00273359,  0.03880827],
       ...,
       [ 0.0352255 ,  0.02467317,  0.01239892, ...,  0.00070752,
         0.01015656,  0.0026823 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.0007054 ,
         0.01747657,  0.008917  ],
       [ 0.00536804,  0.0256284 , -0.02365889, ...,  0.00145341,
         0.07517119,  0.03498576]])

In [16]:
feats = [1,2,5,6,11]

In [17]:
bps, objectives = GGS(_return_df, Kmax = 10, lamb = 1e-4, features = feats)

In [18]:
bp10 = bps[10] # Get breakpoints for K = 10
bp10

[0, 97, 138, 186, 199, 202, 207, 218, 248, 339, 342, 355]

In [19]:
meancovs = GGSMeanCov(_return_df, breakpoints = bp10, lamb = 1e-4, features = feats)

In [20]:
print("Breakpoints are at", bps)
print("Objectives are", objectives)

Breakpoints are at [[0, 355], [0, 97, 355], [0, 97, 210, 355], [0, 97, 202, 210, 355], [0, 97, 202, 210, 339, 355], [0, 97, 138, 201, 210, 339, 355], [0, 97, 138, 186, 202, 210, 339, 355], [0, 97, 138, 186, 202, 210, 339, 342, 355], [0, 97, 138, 186, 202, 210, 248, 339, 342, 355], [0, 97, 138, 186, 202, 207, 218, 248, 339, 342, 355], [0, 97, 138, 186, 199, 202, 207, 218, 248, 339, 342, 355]]
Objectives are [12195.553385804193, 13069.078029955916, 13188.371094112845, 13312.727946566116, 13382.245063684384, 13446.889517750025, 13508.172325165642, 13572.421010725038, 13627.889860383377, 13674.52772810363, 13729.085433654804]
